In [1]:
## CODIGO PARA GUARDAR DATOS DE POSICION DE LOS LANDMARKS EN UN CSV
import cv2
import mediapipe as mp
import csv

# Inicializar MediaPipe para la detección de manos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Crear o abrir el archivo CSV
with open('gestures_data.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    # Escribir encabezado si es un nuevo archivo (21 puntos x, y, y una columna para el gesto)
    writer.writerow([f'x{i}' for i in range(21)] + [f'y{i}' for i in range(21)] + ['gesture'])

    # Capturar video desde la cámara
    cap = cv2.VideoCapture(0)
    gesture = input("Escribe el nombre del primer gesto a capturar: ")  # Solicitar el primer nombre del gesto
    print("Presiona 's' para capturar el gesto, 'n' para cambiar de gesto, y 'q' para salir")

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # Voltear la imagen para un efecto espejo y procesar
        image = cv2.flip(image, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        # Verificar si se detectan manos
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Dibujar los landmarks en la imagen
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Extraer coordenadas de los 21 landmarks
                x_coords = [landmark.x for landmark in hand_landmarks.landmark]
                y_coords = [landmark.y for landmark in hand_landmarks.landmark]
                
                # Mostrar la imagen procesada
                cv2.imshow("Hand Gesture", image)

                # Teclas para guardar datos, cambiar de gesto, o salir
                key = cv2.waitKey(1) & 0xFF
                if key == ord('s'):
                    # Guardar las coordenadas en el CSV junto con el nombre del gesto
                    writer.writerow(x_coords + y_coords + [gesture])
                    print(f"Gesto '{gesture}' capturado y guardado en el CSV.")
                elif key == ord('n'):
                    # Cambiar de gesto
                    gesture = input("Escribe el nuevo nombre del gesto: ")
                elif key == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    break


Presiona 's' para capturar el gesto, 'n' para cambiar de gesto, y 'q' para salir


c:\Users\jufeg\Documents\VSC_codes_IA\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [1]:
## CODIGO DE CREACION DEL MODELO
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Cargar los datos del CSV
data =pd.read_csv('gestures_data.csv', encoding='ISO-8859-1')

# Convertir todas las columnas de X a valores numéricos
for column in data.columns[:-1]:  # Excluir la última columna que es 'gesture'
    data[column] = pd.to_numeric(data[column], errors='coerce')

# Verificar si hay valores NaN después de la conversión
if data.isnull().values.any():
    print("Se encontraron valores no numéricos en las columnas de puntos. Revisar el CSV.")
    data = data.dropna()  # Eliminar filas con valores NaN

# Separar características (X) y etiquetas (y)
X = data.drop('gesture', axis=1).values  # Todas las columnas menos 'gesture'
y = data['gesture'].values               # Solo la columna 'gesture'

# Codificar las etiquetas de texto a números
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)  # Convertir a formato categórico para clasificación

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construir el modelo de red neuronal
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(y.shape[1], activation='softmax')  # Última capa para clasificación con softmax
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}")

# Guardar el modelo entrenado
model.save('gesture_recognition_model.h5')

# Guardar el codificador de etiquetas para usar en predicciones
import joblib
joblib.dump(label_encoder, 'label_encoder.pkl')


Se encontraron valores no numéricos en las columnas de puntos. Revisar el CSV.
Epoch 1/50


c:\Users\jufeg\Documents\VSC_codes_IA\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5680 - loss: 1.1160 - val_accuracy: 0.9003 - val_loss: 0.3306
Epoch 2/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9219 - loss: 0.2658 - val_accuracy: 0.9934 - val_loss: 0.0821
Epoch 3/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9961 - loss: 0.0679 - val_accuracy: 0.9934 - val_loss: 0.0402
Epoch 4/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9934 - loss: 0.0394 - val_accuracy: 0.9801 - val_loss: 0.0340
Epoch 5/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9886 - loss: 0.0416 - val_accuracy: 0.9934 - val_loss: 0.0200
Epoch 6/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9949 - loss: 0.0262 - val_accuracy: 0.9834 - val_loss: 0.0248
Epoch 7/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9933 - loss: 0.0183 - val_accuracy: 0.9934 - val_loss: 0.0135
Epoch 8/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9998 - loss: 0.0070 - val_accuracy: 1.0000 - val_loss: 0.0077
Epo

Precisión en el conjunto de prueba: 1.00


['label_encoder.pkl']

In [2]:
## IMPLEMENTACION DEL MMODELO PARA RECONOCIMIENTO DE GESTOS Y CONEXION SERIAL CON ARDUINO
import cv2
import numpy as np
import tensorflow as tf
import joblib
import mediapipe as mp
import serial
import time
import warnings
import os

# Suprimir advertencias de TensorFlow
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore", category=UserWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Cargar el modelo entrenado y el codificador de etiquetas
model = tf.keras.models.load_model('gesture_recognition_model.h5')
label_encoder = joblib.load('label_encoder.pkl')

# Inicializar Mediapipe
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands()

# Configuración de conexión serial con Arduino
serial_port = 'COM5'  # Ajusta al puerto COM donde esté el Arduino
baud_rate = 9600
ser = serial.Serial(serial_port, baud_rate)
time.sleep(2)  # Tiempo para que se establezca la conexión

# Inicializar la cámara
cap = cv2.VideoCapture(0)

def enviar_angulo(angulo):
    """
    Envía un ángulo al microcontrolador para mover el servo.
    :param angulo: int, ángulo entre 0 y 180
    """
    if 0 <= angulo <= 180:
        ser.write(f'{angulo}\n'.encode())
        print(f"Ángulo enviado: {angulo} grados")
    else:
        print("El ángulo debe estar entre 0 y 180 grados")

def predecir_gesto(landmarks):
    """
    Realiza la predicción del gesto y retorna el nombre del gesto.
    :param landmarks: list, lista de landmarks de la mano
    :return: str, nombre del gesto
    """
    landmarks_array = np.array(landmarks).reshape(1, -1)
    prediction = model.predict(landmarks_array)
    predicted_class = np.argmax(prediction)
    gesture_name = label_encoder.inverse_transform([predicted_class])[0]
    return gesture_name

try:
    while True:
        success, image = cap.read()
        if not success:
            print("No se pudo leer el marco de la cámara.")
            break

        # Convertir la imagen a RGB para Mediapipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Dibujar los landmarks
                mp_draw.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extraer los landmarks en formato de lista con 5 decimales
                landmarks = []
                for landmark in hand_landmarks.landmark:
                    landmarks.append(round(landmark.x, 5))
                    landmarks.append(round(landmark.y, 5))

                # Comprobar la longitud de los landmarks
                if len(landmarks) != 42:
                    raise ValueError(f"Se esperaban 42 valores, pero se recibieron {len(landmarks)}.")
                
                try:
                    # Predecir el gesto
                    gesture_name = predecir_gesto(landmarks)
                    cv2.putText(image, f"Gesto: {gesture_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    # Enviar ángulo específico al Arduino según el gesto detectado
                    if gesture_name == "mano_abierta":
                        enviar_angulo(0)   # Ejemplo: ángulo 0
                    elif gesture_name == "paz":
                        enviar_angulo(90)  # Ejemplo: ángulo 90
                    elif gesture_name == "rock":
                        enviar_angulo(180)  # Ejemplo: ángulo 180

                except Exception as e:
                    cv2.putText(image, f"Ocurrió un error: {e}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    print("Error:", e)

        else:
            cv2.putText(image, "Esperando detección de mano...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Mostrar la imagen con los resultados
        cv2.imshow('Detección de Gestos', image)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Liberar la captura y cerrar conexiones
    cap.release()
    cv2.destroyAllWindows()
    ser.close()
    print("Conexión cerrada.")


SerialException: could not open port 'COM5': FileNotFoundError(2, 'El sistema no puede encontrar el archivo especificado.', None, 2)